In [63]:
import numpy as np
import threading
import time
import pickle
import matplotlib.pyplot as plt
import pandas as pd

In [24]:
def pre_process(shape:int):
    matrix_shape = (shape, shape)
    A = np.random.randint(0, 100, matrix_shape)
    B = np.random.randint(0, 100, matrix_shape)
    return A, B

In [25]:
matrix_mult_for_np = lambda A, B: A @ B

In [26]:
def matrix_mult_single_thread(A, B):
    C = np.zeros((A.shape[0], B.shape[1]), dtype=np.int32)
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                C[i,j] += A[i,k] * B[k,j]
    return C

In [27]:
def matrix_mult_multi_thread(A, B):
    C = np.zeros((A.shape[0], B.shape[1]), dtype=np.int32)
    threading_list = []

    def thread_func(row):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                C[row,j] += A[row,k] * B[k,j]

    for row in range(A.shape[0]):
        t = threading.Thread(target=thread_func, args=(row,))
        t.start()
        threading_list.append(t)
        
    for t in threading_list:
        t.join()
        
    return C

In [28]:
A, B = pre_process(5)
print("A Matrix")
print(A)
print(10*"=")
print("B Matrix")
print(B)

A Matrix
[[58 60 99 97 53]
 [ 4  0  3 44 92]
 [ 5 64 33 10 26]
 [87 16 97 26 30]
 [66 60 52 32 94]]
B Matrix
[[27 52 68 96 76]
 [84 23 88 68 42]
 [ 2 55 19 61 32]
 [21 75  7 11 84]
 [71 95 42 54 33]]


In [29]:
matrix_mult_for_np(A, B)

array([[12604, 22151, 14010, 19616, 19993],
       [ 7570, 12413,  4501,  6019,  7132],
       [ 7633,  6767,  7761,  8359,  5822],
       [ 6563, 15027, 10609, 17263, 13562],
       [14272, 19002, 14928, 19016, 14990]])

In [30]:
matrix_mult_single_thread(A, B)

array([[12604, 22151, 14010, 19616, 19993],
       [ 7570, 12413,  4501,  6019,  7132],
       [ 7633,  6767,  7761,  8359,  5822],
       [ 6563, 15027, 10609, 17263, 13562],
       [14272, 19002, 14928, 19016, 14990]])

In [31]:
matrix_mult_multi_thread(A, B)

array([[12604, 22151, 14010, 19616, 19993],
       [ 7570, 12413,  4501,  6019,  7132],
       [ 7633,  6767,  7761,  8359,  5822],
       [ 6563, 15027, 10609, 17263, 13562],
       [14272, 19002, 14928, 19016, 14990]])

In [32]:
A, B = pre_process(200)

In [33]:
%%time
C = matrix_mult_for_np(A, B)

CPU times: total: 0 ns
Wall time: 5.52 ms


In [34]:
%%time
 C = matrix_mult_multi_thread(A, B)

CPU times: total: 375 ms
Wall time: 4.09 s


In [35]:
%%time
C = matrix_mult_single_thread(A, B)

CPU times: total: 3.33 s
Wall time: 3.82 s


In [36]:
def calculate_elapsed_time(A, B, func):
    start = time.time()
    func(A,B)
    end = time.time()
    return end - start

In [37]:
result = {}

In [38]:
for i in range(50, 501, 50):
    A, B = pre_process(i)
    single_thread_time = calculate_elapsed_time(A, B, matrix_mult_single_thread)
    multi_thread_time = calculate_elapsed_time(A, B, matrix_mult_multi_thread)
    result[f"{i}x{i}"] = [single_thread_time, multi_thread_time]
    print(f"Matrix size: {i}x{i} | single thread: {single_thread_time:.3f} | multi thread: {multi_thread_time:.3f}")

Matrix size: 50x50 | single thread: 0.061 | multi thread: 0.073
Matrix size: 100x100 | single thread: 0.482 | multi thread: 0.513
Matrix size: 150x150 | single thread: 1.601 | multi thread: 1.698
Matrix size: 200x200 | single thread: 3.855 | multi thread: 3.988
Matrix size: 250x250 | single thread: 7.385 | multi thread: 7.672
Matrix size: 300x300 | single thread: 13.129 | multi thread: 13.313
Matrix size: 350x350 | single thread: 20.327 | multi thread: 21.041
Matrix size: 400x400 | single thread: 30.333 | multi thread: 31.396
Matrix size: 450x450 | single thread: 43.184 | multi thread: 44.785
Matrix size: 500x500 | single thread: 59.293 | multi thread: 61.603


In [39]:
result

{'50x50': [0.06145310401916504, 0.07293844223022461],
 '100x100': [0.4822709560394287, 0.5128941535949707],
 '150x150': [1.6010997295379639, 1.6979968547821045],
 '200x200': [3.855052947998047, 3.988065719604492],
 '250x250': [7.385160446166992, 7.672171354293823],
 '300x300': [13.128905773162842, 13.312986135482788],
 '350x350': [20.326743841171265, 21.040990591049194],
 '400x400': [30.333247661590576, 31.39557456970215],
 '450x450': [43.18406319618225, 44.78542399406433],
 '500x500': [59.29327940940857, 61.602596044540405]}

In [40]:
with open("result.pkl", "wb") as f:
    pickle.dump(result, f)